In [18]:
import pandas as pd 
import os
import re

In [ ]:
PATH = r"/Users/jimmyhe/Desktop/ML/KaggleCompetitions/NCAA/MasseyRating/"
OUTPUT_PATH = r'/Users/jimmyhe/Desktop/ML/KaggleCompetitions/NCAA/MasseyCleaned/'
common_columns = None

for filename in os.listdir(PATH):
    if filename == '.DS_Store': 
        continue
    filepath = os.path.join(PATH, filename)
    
    if os.path.isfile(filepath):
        temp = pd.read_csv(filepath)
        temp = temp.drop(temp.columns[[1, 2, 3, 4, 5]], axis=1)
        
        if common_columns is None:
            common_columns = set(temp.columns)  # Initialize with columns of the first file
        else:
            common_columns &= set(temp.columns)  # Intersect with columns of subsequent files

# Second pass: Process each file using the identified common columns
for filename in os.listdir(PATH):
    if filename == '.DS_Store': 
        continue
    filepath = os.path.join(PATH, filename)
    
    if os.path.isfile(filepath):
        print(f"Processing {filename}")
        temp = pd.read_csv(filepath)        
        df_filtered = temp[list(common_columns)].copy()
        
        # Optionally, add the 'Season' column to track the season
        season_match = re.search(r"_(\d{4})_", filepath)
        if season_match:
            season = season_match.group(1)
            df_filtered['Season'] = int(season)
        
        print(f"Shape of the filtered DataFrame for {filename}:", df_filtered.shape)
        
        output_filename = f"filtered_{filename}" if season_match else f"filtered_{filename}"
        df_filtered.to_csv(os.path.join(OUTPUT_PATH, output_filename), index=False)


Processing Massey_2022_03_13.csv
Shape of the filtered DataFrame for Massey_2022_03_13.csv: (358, 24)
Processing Massey_2024_03_17.csv
Shape of the filtered DataFrame for Massey_2024_03_17.csv: (362, 24)
Processing Massey_2023_03_12.csv
Shape of the filtered DataFrame for Massey_2023_03_12.csv: (363, 24)
Processing Massey_2025_03_16.csv
Shape of the filtered DataFrame for Massey_2025_03_16.csv: (364, 24)
Processing Massey_2018_03_11.csv
Shape of the filtered DataFrame for Massey_2018_03_11.csv: (351, 24)
Processing Massey_2021_03_14.csv
Shape of the filtered DataFrame for Massey_2021_03_14.csv: (347, 24)
Processing Massey_2019_03_17.csv
Shape of the filtered DataFrame for Massey_2019_03_17.csv: (353, 24)


In [45]:
import os
import pandas as pd
import re

PATH = r"/Users/jimmyhe/Desktop/ML/KaggleCompetitions/NCAA/MasseyRating/"
OUTPUT_PATH = r'/Users/jimmyhe/Desktop/ML/KaggleCompetitions/NCAA/MasseyCleaned/'

common_columns = None

# First pass: Identify common columns across all files
for filename in os.listdir(PATH):
    if filename == '.DS_Store': 
        continue
    filepath = os.path.join(PATH, filename)
    
    if os.path.isfile(filepath):
        temp = pd.read_csv(filepath)
        temp = temp.drop(temp.columns[[1, 2, 3, 4, 5]], axis=1)
        
        if common_columns is None:
            common_columns = set(temp.columns)  # Initialize with columns of the first file
        else:
            common_columns &= set(temp.columns)  # Intersect with columns of subsequent files

# Second pass: Process each file using the identified common columns
for filename in os.listdir(PATH):
    if filename == '.DS_Store': 
        continue
    filepath = os.path.join(PATH, filename)
    
    if os.path.isfile(filepath):
        print(f"Processing {filename}")
        temp = pd.read_csv(filepath)        
        df_filtered = temp[list(common_columns)].copy()
        
        # Optionally, add the 'Season' column to track the season
        season = None
        season_match = re.search(r"_(\d{4})_", filepath)
        if season_match:
            season = season_match.group(1)
            df_filtered['Season'] = int(season)
        
        # Ensure "Team" and "Season" are the first columns in the filtered DataFrame
        columns = ['Team', 'Season'] + [col for col in common_columns if col not in ['Team', 'Season']]
        df_filtered = df_filtered[columns]
        
        # Debug: Print the shape of the filtered DataFrame
        print(f"Shape of the filtered DataFrame for {filename}: {df_filtered.shape}")
        
        # Set output filename based on season presence
        output_filename = f"filtered_{filename}"
        
        # Save the filtered DataFrame to the output path
        df_filtered.to_csv(os.path.join(OUTPUT_PATH, output_filename), index=False)


Processing Massey_2022_03_13.csv
Shape of the filtered DataFrame for Massey_2022_03_13.csv: (358, 24)
Processing Massey_2024_03_17.csv
Shape of the filtered DataFrame for Massey_2024_03_17.csv: (362, 24)
Processing Massey_2023_03_12.csv
Shape of the filtered DataFrame for Massey_2023_03_12.csv: (363, 24)
Processing Massey_2025_03_16.csv
Shape of the filtered DataFrame for Massey_2025_03_16.csv: (364, 24)
Processing Massey_2018_03_11.csv
Shape of the filtered DataFrame for Massey_2018_03_11.csv: (351, 24)
Processing Massey_2021_03_14.csv
Shape of the filtered DataFrame for Massey_2021_03_14.csv: (347, 24)
Processing Massey_2019_03_17.csv
Shape of the filtered DataFrame for Massey_2019_03_17.csv: (353, 24)


In [47]:

Cleaned_path = r"/Users/jimmyhe/Desktop/ML/KaggleCompetitions/NCAA/MasseyCleaned/"
Massey_2025 = pd.read_csv(os.path.join(Cleaned_path, 'filtered_Massey_2025_03_16.csv'))
reference_columns = Massey_2025.columns  # Use the columns from Massey_2025 as the reference

# Check if the directory exists
if not os.path.exists(Cleaned_path):
    raise FileNotFoundError(f"The directory {Cleaned_path} does not exist.")

# List to store DataFrames for vertical concatenation
dataframes = []

# Iterate through files in the directory
for filename in os.listdir(Cleaned_path):
    if filename == '.DS_Store':  # Skip system files like .DS_Store on macOS
        continue
    filepath = os.path.join(Cleaned_path, filename)
    
    # Read each CSV file
    print(f"Processing file: {filepath}")
    temp = pd.read_csv(filepath)
    print(f"Shape of {filename}: {temp.shape}")
    
    # Reorder columns to match the reference column order
    temp = temp.reindex(columns=reference_columns)
    
    # Append the reordered DataFrame to the list
    dataframes.append(temp)

# Concatenate all DataFrames vertically (stacking rows)
joined_df = pd.concat(dataframes, axis=0, ignore_index=True)  # `ignore_index=True` resets row indices

print(f"Shape of the joined DataFrame: {joined_df.shape}")

# Save the joined DataFrame to a new CSV file
output_path = os.path.join(Cleaned_path, "joined_Massey_Rating.csv")
joined_df.to_csv(output_path, index=False)
print(f"Joined DataFrame saved to: {output_path}")
print('Final shape:', joined_df.shape)


Processing file: /Users/jimmyhe/Desktop/ML/KaggleCompetitions/NCAA/MasseyCleaned/filtered_Massey_2021_03_14.csv
Shape of filtered_Massey_2021_03_14.csv: (347, 24)
Processing file: /Users/jimmyhe/Desktop/ML/KaggleCompetitions/NCAA/MasseyCleaned/filtered_Massey_2019_03_17.csv
Shape of filtered_Massey_2019_03_17.csv: (353, 24)
Processing file: /Users/jimmyhe/Desktop/ML/KaggleCompetitions/NCAA/MasseyCleaned/filtered_Massey_2018_03_11.csv
Shape of filtered_Massey_2018_03_11.csv: (351, 24)
Processing file: /Users/jimmyhe/Desktop/ML/KaggleCompetitions/NCAA/MasseyCleaned/filtered_Massey_2023_03_12.csv
Shape of filtered_Massey_2023_03_12.csv: (363, 24)
Processing file: /Users/jimmyhe/Desktop/ML/KaggleCompetitions/NCAA/MasseyCleaned/filtered_Massey_2025_03_16.csv
Shape of filtered_Massey_2025_03_16.csv: (364, 24)
Processing file: /Users/jimmyhe/Desktop/ML/KaggleCompetitions/NCAA/MasseyCleaned/filtered_Massey_2022_03_13.csv
Shape of filtered_Massey_2022_03_13.csv: (358, 24)
Processing file: /User

In [48]:
joined_df['Season'].unique()

array([2021, 2019, 2018, 2023, 2025, 2022, 2024])

In [56]:
joined_df.rename(columns={'Team': 'TeamName'}, inplace=True)
joined_df.shape


(2498, 24)

In [58]:
Comp_data_PATH = r"/Users/jimmyhe/Desktop/ML/KaggleCompetitions/NCAA/march-machine-learning-mania-2025/"
M_teams = pd.read_csv(Comp_data_PATH + 'MTeams.csv')
M_teams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2025
1,1102,Air Force,1985,2025
2,1103,Akron,1985,2025
3,1104,Alabama,1985,2025
4,1105,Alabama A&M,2000,2025


In [59]:
M_teams[M_teams["TeamName"] == "Michigan"]

,TeamID,TeamName,FirstD1Season,LastD1Season
175,1276,Michigan,1985,2025


In [61]:
merged_df = pd.merge(joined_df, M_teams, on='TeamName', how='inner')

# Check for unmatched TeamNames in df1
unmatched_in_df1 = set(joined_df['TeamName']) - set(M_teams['TeamName'])
if unmatched_in_df1:
    raise ValueError(f"Unmatched TeamNames in the first DataFrame: {unmatched_in_df1}")

# Check for unmatched TeamNames in df2
unmatched_in_df2 = set(M_teams['TeamName']) - set(joined_df['TeamName'])
if unmatched_in_df2:
    print(f"Unmatched TeamNames in the second DataFrame: {unmatched_in_df2}")


Unmatched TeamNames in the second DataFrame: {'Armstrong St', 'W Texas A&M', 'Brooklyn', 'NE Illinois', 'Alliant Intl', 'Centenary', 'Birmingham So', 'Hardin-Simmons', 'Morris Brown', 'Augusta', 'W Salem St', 'Okla City', 'Utica'}


In [68]:
TNames_Massey = joined_df['TeamName'].unique()
TNames_Comp = M_teams['TeamName'].unique()
TNames_Massey.sort()
print("TeamNames_Massey:")
print(list(TNames_Massey))

print("\nTeamNames_Comp:")
print(list(TNames_Comp))

TeamNames_Massey:
['Abilene Chr', 'Air Force', 'Akron', 'Alabama', 'Alabama A&M', 'Alabama St', 'Alcorn St', 'American Univ', 'Appalachian St', 'Arizona', 'Arizona St', 'Ark Little Rock', 'Ark Pine Bluff', 'Arkansas', 'Arkansas St', 'Army', 'Auburn', 'Austin Peay', 'BYU', 'Ball St', 'Baylor', 'Bellarmine', 'Belmont', 'Bethune-Cookman', 'Binghamton', 'Boise St', 'Boston College', 'Boston Univ', 'Bowling Green', 'Bradley', 'Brown', 'Bryant', 'Bucknell', 'Buffalo', 'Butler', 'C Michigan', 'CS Bakersfield', 'CS Fullerton', 'CS Northridge', 'CS Sacramento', 'Cal Baptist', 'Cal Poly', 'California', 'Campbell', 'Canisius', 'Cent Arkansas', 'Central Conn', 'Charleston So', 'Charlotte', 'Chattanooga', 'Chicago St', 'Cincinnati', 'Citadel', 'Clemson', 'Cleveland St', 'Coastal Car', 'Col Charleston', 'Colgate', 'Colorado', 'Colorado St', 'Columbia', 'Connecticut', 'Coppin St', 'Cornell', 'Creighton', 'Dartmouth', 'Davidson', 'Dayton', 'DePaul', 'Delaware', 'Delaware St', 'Denver', 'Detroit', 'Dra

In [72]:
len(TNames_Massey), len(TNames_Comp)

(367, 380)

In [71]:
common_items = list(set(TNames_Massey) & set(TNames_Comp))
len(common_items)


367

In [73]:
unique_in_sequence_one = list(set(TNames_Massey) - set(TNames_Comp))
len(unique_in_sequence_one)

0

Makes sure that the Massey Rating has the exact same format as teamanmes in the kaggle comp 

In [79]:
tmp = joined_df.merge(M_teams, on='TeamName', how='left')
tmp.head()

,TeamName,Season,POM,WIL,JNG,7OT,MAS,JJK,EBP,FAS,...,TRP,DII,MOR,TRK,KPK,RT,WLK,TeamID,FirstD1Season,LastD1Season
0,Gonzaga,2021,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1211,1985,2025
1,Baylor,2021,4,2,2,19,3,2,2,2,...,2,2,3,3,2,3,3,1124,1985,2025
2,Illinois,2021,3,5,3,18,2,5,4,3,...,3,3,2,4,3,8,2,1228,1985,2025
3,Michigan,2021,2,4,4,9,4,3,6,4,...,6,4,4,5,4,5,4,1276,1985,2025
4,Houston,2021,6,9,12,4,6,4,3,5,...,5,6,6,2,5,2,6,1222,1985,2025


In [80]:
final_train = tmp[['TeamName','TeamID', 'Season', 'FirstD1Season',
       'LastD1Season','POM', 'WIL', 'JNG', '7OT', 'MAS', 'JJK', 'EBP',
       'FAS', 'HAS', 'DOK', 'LMC', 'USA', 'PGH', 'BBT', 'BIH', 'TRP', 'DII',
       'MOR', 'TRK', 'KPK', 'RT', 'WLK',]]

In [87]:
final_train = final_train.sort_values(by='Season', ascending=False)
final_train.to_csv('/Users/jimmyhe/Desktop/ML/KaggleCompetitions/NCAA/Train_Set/Massey_final.csv')